# TV Script Generation
This project generates [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs, especially LSTMs. It'll use part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).

In [5]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Removes unnecassary part of data
text = text[81:]

### Data Exploration

In [6]:
import numpy as np

view_sentence_range = (0, 7)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 7:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.


- Creating dictionary to go from the words to an id: `vocab_to_int`
- Creating dictionary to go from the id to word: `int_to_vocab`

In [7]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: i + 1 for i, word in enumerate(vocab)}
    int_to_vocab = {i + 1: word for i, word in enumerate(vocab)}
    
    return vocab_to_int, int_to_vocab

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

This dictionary will be used to token the symbols like "!" into "||Exclamation_Mark||".  

In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    d = {'.': '||Period||',  ',': '||Comma||',  '"': '||Quotation_Mark||', 
         ';': '||Semicolon||',  '!': '||Exclamation_Mark||',  '?': '||Question_mark||',
         '(': '||Left_Parentheses||',  ')': '||Right_Parentheses||',  '--': '||Dash||', '\n': '||Return||'}
    return d


In [9]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

## Check Point for Preprocessed Data

In [14]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [15]:
list(int_to_vocab.items())[:9]

[(1, '||period||'),
 (2, '||return||'),
 (3, '||comma||'),
 (4, '||left_parentheses||'),
 (5, '||right_parentheses||'),
 (6, 'the'),
 (7, 'i'),
 (8, 'you'),
 (9, '||exclamation_mark||')]

### Checking the Version of TensorFlow and Access to GPU

In [17]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.6.0


/Users/hakobtamazyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # This is added back by InteractiveShellApp.init_path()


### Building the Neural Network

In [18]:
def get_inputs():
    
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='targets')
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    KeepProb = tf.placeholder(tf.float32, name='keep_prob')
    
    return Input, Targets, LearningRate, KeepProb


In [30]:
def get_init_cell(batch_size, num_layers, rnn_size, keep_prob):
    
    def build_cell(lstm_size, keep_prob):

        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    Cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    InitialState = tf.identity(Cell.zero_state(batch_size, tf.float32), name='initial_state')
    return Cell, InitialState


In [31]:
def get_embed(input_data, vocab_size, embed_dim):

    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


In [32]:
def build_rnn(cell, inputs):
    
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name='final_state')
    
    return Outputs, FinalState


In [33]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    
    embed = get_embed(input_data, vocab_size, embed_dim)
    Outputs, FinalState = build_rnn(cell, embed)
    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size, activation_fn=None)
    
    return Logits, FinalState


### Batches

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

In [34]:
def get_batches(int_text, batch_size, seq_length):
    
    chars_per_batch = batch_size * seq_length
    n_batches = len(int_text)//chars_per_batch
    
    int_text = np.array(int_text)
    arr = int_text[:n_batches * chars_per_batch]
    
    arr = arr.reshape((batch_size, -1))
    batches = []
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        yy = arr[:, n+1:n+seq_length+1]
        
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:yy.shape[1]] = yy
        
        batches.append([x, y])
    
    return np.array(batches)


### Hyperparameters

In [35]:
num_epochs = 80
batch_size = 120
rnn_size = 512
embed_dim = 300
seq_length = 32
learning_rate = 0.01
num_layers = 1
keep_prob = 1.0

save_dir = './save'

### Building the Graph

In [42]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr, keep = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], num_layers, rnn_size, keep)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train

In [43]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate,
                keep: keep_prob}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

        print('Epoch {:>3}  train_loss = {:.3f}'.format(epoch_i, train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0  train_loss = 5.708
Epoch   1  train_loss = 4.911
Epoch   2  train_loss = 4.322
Epoch   3  train_loss = 3.822
Epoch   4  train_loss = 3.399
Epoch   5  train_loss = 3.018
Epoch   6  train_loss = 2.680
Epoch   7  train_loss = 2.390
Epoch   8  train_loss = 2.064
Epoch   9  train_loss = 1.808
Epoch  10  train_loss = 1.603
Epoch  11  train_loss = 1.458
Epoch  12  train_loss = 1.316
Epoch  13  train_loss = 1.188
Epoch  14  train_loss = 0.984
Epoch  15  train_loss = 0.846
Epoch  16  train_loss = 0.688
Epoch  17  train_loss = 0.597
Epoch  18  train_loss = 0.494
Epoch  19  train_loss = 0.431
Epoch  20  train_loss = 0.369
Epoch  21  train_loss = 0.311
Epoch  22  train_loss = 0.258
Epoch  23  train_loss = 0.227
Epoch  24  train_loss = 0.192
Epoch  25  train_loss = 0.163
Epoch  26  train_loss = 0.146
Epoch  27  train_loss = 0.130
Epoch  28  train_loss = 0.122
Epoch  29  train_loss = 0.111
Epoch  30  train_loss = 0.105
Epoch  31  train_loss = 0.098
Epoch  32  train_loss = 0.097
Epoch  33 

In [51]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

## Checkpoint

In [1]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

/Users/hakobtamazyan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_tensors(loaded_graph):
    
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    KeepProbTensor = loaded_graph.get_tensor_by_name('keep_prob:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return InputTensor, KeepProbTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


In [3]:
def pick_word(probabilities, int_to_vocab, top_n=4):
    
    p = np.squeeze(probabilities)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    word_id = np.random.choice(len(p), p=p)

    return int_to_vocab[word_id]


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [4]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, keep, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, keep: 1.0, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[:, dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print()
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save

moe_szyslak:(stunned) nigeria?
moe_szyslak:(repressed rage) homer, can i speak to you in private?
homer_simpson: can i try it?
moe_szyslak: eh, this is serious. this bashir kid is muslim, and therefore up to the pool table.
little_hibbert_girl: daddy, this place smells like tinkle.
dr. _julius_hibbert: mmm-hmm, i'm going to use your tongue to paint my boat.
waylon_smithers: uh, hello, is there anything in?
moe_szyslak: uh, let me check the lost and found.
moe_szyslak: what do we got to do something to change his war.
lenny_leonard: that's right, brainiac. you cost us our jobs, which we need for workin'.
homer_simpson: hi, moe.
moe_szyslak: okay, homer, i was so self-centered. no wonder i didn't see it coming.(shaking head) that's how it is, though: a minute. i gotta get back to the back of...
barney_gumble: you know, i heard the jackpot's up to one hundred and thirty million dollars.
